In [79]:
import os
from glob import glob
import pandas as pd
import numpy as np

In [106]:
import re
import shutil

In [81]:
os.getcwd()

'E:\\transceiver_project\\time lapse propagation 7 15 20'

In [82]:
home_dir = "E:/transceiver_project/time lapse propagation 7 15 20/"
if os.path.exists(home_dir):
    os.chdir(home_dir)
else:
    print("Directory not found")

In [83]:
os.chdir(home_dir)
# os.chdir("from 88 h/Group/")

In [133]:
vids = glob("from 0h/*_?_*")
vids_0 = sorted([v for v in vids if v.endswith("0h")])
vids_88 = sorted([v for v in vids if v.endswith("88h")])
paired_vids = list(zip(vids_0, vids_88))

In [138]:
a, b = paired_vids[0]
glob(b + "/T*");

In [157]:
shutil.rmtree('from 0h/gone')

In [165]:
for v0, v88 in paired_vids:
#     t_from_88 = glob(v88 + "/T*")
#     for t in t_from_88:
#         shutil.move(os.path.join(v88, t), v0)
#     shutil.rmtree(v88)
    os.rename(v0, "_".join(v0.split("_")[:2]))

In [ ]:
for v0, v88 in paired_vids[1:]:
    t_from_88 = glob(v88 + "/T*")
    print(t_from_88)
    for t in t_from_88:
        t = os.path.split(t)[-1]
        print(os.path.join(v88, t))
        shutil.move(os.path.join(v88, t), v0)

In [60]:
densities = [1000, 2500, 5000, 10000, 20000, 40000, 80000, 160000]
densities = [str(s) for s in densities]

In [61]:
vid_names = ["_".join([d, l]) for d in densities for l in ("A", "B", "C")]

<hr>

Here let's turn these individual TIFFs into TIFF stacks

In [267]:
import imageio

import os
import numpy as np

In [268]:
os.getcwd()

'E:\\transceiver_project\\20200715_time_lapse_propagation\\time_lapse_20x'

In [269]:
home_dir = "E:/transceiver_project/20200715_time_lapse_propagation/time_lapse_20x/"
if os.path.exists(home_dir):
    os.chdir(home_dir)
else:
    print("Directory not found")

In [270]:
os.chdir(home_dir)

In [217]:
vids = glob("*_*")

In [218]:
vids = vids[:1]

In [219]:
vids

['0_test']

In [211]:
for v in vids:
    ims = glob(os.path.join(v, "*/*.tif"))
    new_ims = [im.split("_") for im in ims]
    new_ims = ["_".join([*s[:3], v, *s[4:]]) for s in new_ims]
#     print(new_ims)
    for i, _ in enumerate(ims):
        os.rename(ims[i], new_ims[i])

In [225]:
image = imageio.imread("0test/T0001/Image_T0001_40000_B_CH4.tif")

In [181]:
image.shape

(1440, 1920)

In [ ]:
fi.write_multipage(image, 'multipage.tif')

In [292]:
import cv2
import os

image_folder = '5000_C'
video_name = '5000 cells C.avi'
fps = 10

images = glob(image_folder + "/*/*Overlay.tif")
frame = cv2.imread(images[0])
height, width, layers = frame.shape
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"MP42"), fps, (width,height))
for image in images:
    video.write(cv2.imread(image))

cv2.destroyAllWindows()
video.release()

<hr>

Let's draw ROIs for all the videos so that we can analyze signal propagation dynamics.

For each video, I have decided using some criteria whether or not to analyze it with respect to signal propagation. A time-lapse was rejected for propagation analysis if
- there were too few cells (leading to high variance in cell-wise metrics and inherently stochastic behaviors such as motility)
- the edge of the well impinges on the field of view
- there are other loci of signal propagation in addition to the one being studied, and the borders are difficult to delineate

The outcome of these exclusion criteria is encoded as a Boolean entry in the metadata column `can_draw_ROI`.

In [239]:
vids = glob("*_*")

In [240]:
df = pd.DataFrame(dict(video_folder=vids))

df["plating_density"] = [s.split("_")[0] for s in df.video_folder.values]
df["replicate"] = [s.split("_")[1] for s in df.video_folder.values]
df = df.sort_values(["plating_density", "replicate"])

In [248]:
can_draw_ROI = np.array(
    [
        False,
        False,
        False,
        True,
        False,
        False,
        False,
        True,
        False,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        False,
        True,
    ],
    dtype=bool,
)

In [245]:
can_draw_ROI.size

23

In [246]:
df.shape

(24, 3)

In [242]:
df

,video_folder,plating_density,replicate
3,1000_A,1000,A
4,1000_B,1000,B
5,1000_C,1000,C
0,10000_A,10000,A
1,10000_B,10000,B
2,10000_C,10000,C
6,160000_A,160000,A
7,160000_B,160000,B
8,160000_C,160000,C
9,20000_A,20000,A


In [66]:
os.chdir(home_dir)
os.chdir("from 0h")

In [60]:
densities = [1000, 2500, 5000, 10000, 20000, 40000, 80000, 160000]
densities = [str(s) for s in densities]

In [61]:
vid_names = ["_".join([d, l]) for d in densities for l in ("A", "B", "C")]